In [17]:
from train_tf import *
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from parameters import *
import socket
import tensorflow as tf
import shap

import datetime
import os

# Calculate PFI
import numpy as np
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score

In [9]:
# Adjust the path to where your model is stored
load_dir ='/Users/ruoqig/punim2039/EightK/res/temp/vec_pred/8f0a05c1bb38a954d7defbe0ae7e0fb3725f1c97236de7c82f8f002924b05c2e/OPT_125m/NEWS_SINGLE/'
model_outputs = [os.path.join(load_dir, f) for f in os.listdir(load_dir) if re.match(r'\d+.p_model.keras$', f)]
model_outputs

['/Users/ruoqig/punim2039/EightK/res/temp/vec_pred/8f0a05c1bb38a954d7defbe0ae7e0fb3725f1c97236de7c82f8f002924b05c2e/OPT_125m/NEWS_SINGLE/2014.p_model.keras',
 '/Users/ruoqig/punim2039/EightK/res/temp/vec_pred/8f0a05c1bb38a954d7defbe0ae7e0fb3725f1c97236de7c82f8f002924b05c2e/OPT_125m/NEWS_SINGLE/2013.p_model.keras',
 '/Users/ruoqig/punim2039/EightK/res/temp/vec_pred/8f0a05c1bb38a954d7defbe0ae7e0fb3725f1c97236de7c82f8f002924b05c2e/OPT_125m/NEWS_SINGLE/2017.p_model.keras',
 '/Users/ruoqig/punim2039/EightK/res/temp/vec_pred/8f0a05c1bb38a954d7defbe0ae7e0fb3725f1c97236de7c82f8f002924b05c2e/OPT_125m/NEWS_SINGLE/2018.p_model.keras',
 '/Users/ruoqig/punim2039/EightK/res/temp/vec_pred/8f0a05c1bb38a954d7defbe0ae7e0fb3725f1c97236de7c82f8f002924b05c2e/OPT_125m/NEWS_SINGLE/2016.p_model.keras',
 '/Users/ruoqig/punim2039/EightK/res/temp/vec_pred/8f0a05c1bb38a954d7defbe0ae7e0fb3725f1c97236de7c82f8f002924b05c2e/OPT_125m/NEWS_SINGLE/2012.p_model.keras',
 '/Users/ruoqig/punim2039/EightK/res/temp/vec_pred/8

In [13]:
# Load the optimized model
model = tf.keras.models.load_model(model_outputs[0])

In [2]:
"""
Training the model.
"""
par = get_main_experiments(7, train_gpu=True, tensorboard=True)
par.enc.opt_model_type = OptModelType.OPT_125m
par.enc.news_source = NewsSource.NEWS_SINGLE

# Training args
par.train.use_tf_models = True
par.train.l1_ratio = [0.5]
par.train.batch_size = 128
par.train.monitor_metric = 'val_auc'
par.train.patience = 3
par.train.max_epoch = 3
par.train.abny = False

# par.train.T_train = 1


# Skip Normalisation
# par.train.norm = None

start = time.time()

temp_save_dir = par.get_res_dir()
print(temp_save_dir, flush=True)
already_processed = os.listdir(temp_save_dir)
save_name = f'{par.grid.year_id}.p'
print('save_name', save_name, "already_processed", already_processed)
if save_name in already_processed:
    print(f'Already processed {save_name}', flush=True)
else:
    trainer = PipelineTrainer(par)
    trainer.def_create_the_datasets(
        filter_func=lambda x: 'mean' in x.split('/')[-1].split('_'),  # filter by 'mean' in file name
    )
    print('Preprocessing time', np.round((time.time() - start) / 60, 5), 'min', flush=True)

model = trainer.train_model(tr_data=trainer.train_val_dataset, val_data=None, reg_to_use=tf.keras.regularizers.l1_l2(0.001, 0.001))

model1_evaluation_results_val = model.evaluate(
    trainer.val_dataset,
    return_dict=True
)

model1_evaluation_results_test = model.evaluate(
    trainer.test_dataset,
    return_dict=True
)
model1_evaluation_results_test

trainer.model = model
df = trainer.get_prediction_on_test_sample()
df.to_pickle(temp_save_dir + save_name)
par.save(temp_save_dir)

START WORKING ON  7
comb 8 / 264
/data/gpfs/projects/punim2119/EightK/res/temp/vec_pred/d7cc8fc49ceec47bf5ac309008d70a2ed2e1e7df992a46af4c24ecdaf9a71c85/OPT_125m/NEWS_SINGLE/
save_name 2019.p already_processed []
tensorboard --logdir=C:\Users\nxcle\repo\EightK\logs\fit\20240129-234353
Preprocessing time 0.01687 min


START WORKING ON  7
comb 8 / 264
/Users/ruoqig/punim2039/EightK/res/temp/vec_pred/d7cc8fc49ceec47bf5ac309008d70a2ed2e1e7df992a46af4c24ecdaf9a71c85/OPT_125m/NEWS_SINGLE/
save_name 2019.p already_processed []
tensorboard --logdir=/Users/ruoqig/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/coding-projects/phd/antoine/EightK/logs/fit/20240307-221048
Preprocessing time 0.00266 min


2024-03-07 22:10:48.920211: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-03-07 22:10:48.920224: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-03-07 22:10:48.921133: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


NameError: name 'regularizers' is not defined